# Ассоциативные правила

Ассоциативные правила (или ассоциативные правила выявления зависимостей) — это метод машинного обучения, используемый для поиска скрытых взаимосвязей между элементами в больших наборах данных. Этот метод часто применяется в задачах анализа корзины покупок, когда цель — выявить, какие товары покупаются вместе.  

**Основные концепции:**  
1. Поддержка (Support) — процент транзакций, в которых встречаются оба элемента правила. 
Например, если 30% покупателей купили молоко и хлеб, то поддержка этого набора — 30%.  

2. Достоверность (Confidence) — вероятность того, что если покупатель приобрёл один товар, то он купит и другой.  
Например, если 70% людей, купивших молоко, также покупают хлеб, то достоверность правила «молоко → хлеб» равна 70%.  

3. Подъем (Lift) — показатель, который сравнивает, насколько часто два товара встречаются вместе, по сравнению с тем, если бы они были независимы.  
Подъем больше 1 говорит о том, что товары связаны положительно.  

**Пример:**  
Правило «молоко → хлеб» с поддержкой 20% и достоверностью 70% означает, что 20% всех транзакций включают оба товара, и 70% из тех, кто купил молоко, также купили хлеб.  

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
%matplotlib inline

Возьмем [датасет](https://archive.ics.uci.edu/dataset/352/online+retail) о покупках:  

In [2]:
from ucimlrepo import fetch_ucirepo 
  
online_retail = fetch_ucirepo(id=352) 
  
data = online_retail.data.features 

In [3]:
online_retail.metadata


{'uci_id': 352,
 'name': 'Online Retail',
 'repository_url': 'https://archive.ics.uci.edu/dataset/352/online+retail',
 'data_url': 'https://archive.ics.uci.edu/static/public/352/data.csv',
 'abstract': 'This is a transactional data set which contains all the transactions occurring between 01/12/2010 and 09/12/2011 for a UK-based and registered non-store online retail.',
 'area': 'Business',
 'tasks': ['Classification', 'Clustering'],
 'characteristics': ['Multivariate', 'Sequential', 'Time-Series'],
 'num_instances': 541909,
 'num_features': 6,
 'feature_types': ['Integer', 'Real'],
 'demographics': [],
 'target_col': None,
 'index_col': ['InvoiceNo', 'StockCode'],
 'has_missing_values': 'no',
 'missing_values_symbol': None,
 'year_of_dataset_creation': 2015,
 'last_updated': 'Mon Oct 21 2024',
 'dataset_doi': '10.24432/C5BW33',
 'creators': ['Daqing Chen'],
 'intro_paper': {'ID': 361,
  'type': 'NATIVE',
  'title': 'Data mining for the online retail industry: A case study of RFM model

In [4]:
online_retail.variables


,name,role,type,demographic,description,units,missing_values
0,InvoiceNo,ID,Categorical,None,a 6-digit integral number uniquely assigned to...,None,no
1,StockCode,ID,Categorical,None,a 5-digit integral number uniquely assigned to...,None,no
2,Description,Feature,Categorical,None,product name,None,no
3,Quantity,Feature,Integer,None,the quantities of each product (item) per tran...,None,no
4,InvoiceDate,Feature,Date,None,the day and time when each transaction was gen...,None,no
5,UnitPrice,Feature,Continuous,None,product price per unit,sterling,no
6,CustomerID,Feature,Categorical,None,a 5-digit integral number uniquely assigned to...,None,no
7,Country,Feature,Categorical,None,the name of the country where each customer re...,None,no


In [5]:
data

,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,WHITE HANGING HEART T-LIGHT HOLDER,6,12/1/2010 8:26,2.55,17850.0,United Kingdom
1,WHITE METAL LANTERN,6,12/1/2010 8:26,3.39,17850.0,United Kingdom
2,CREAM CUPID HEARTS COAT HANGER,8,12/1/2010 8:26,2.75,17850.0,United Kingdom
3,KNITTED UNION FLAG HOT WATER BOTTLE,6,12/1/2010 8:26,3.39,17850.0,United Kingdom
4,RED WOOLLY HOTTIE WHITE HEART.,6,12/1/2010 8:26,3.39,17850.0,United Kingdom
...,...,...,...,...,...,...
541904,PACK OF 20 SPACEBOY NAPKINS,12,12/9/2011 12:50,0.85,12680.0,France
541905,CHILDREN'S APRON DOLLY GIRL,6,12/9/2011 12:50,2.10,12680.0,France
541906,CHILDRENS CUTLERY DOLLY GIRL,4,12/9/2011 12:50,4.15,12680.0,France
541907,CHILDRENS CUTLERY CIRCUS PARADE,4,12/9/2011 12:50,4.15,12680.0,France


In [6]:
data_info = data.info()
data_head = data.head()

data_info, data_head

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 541909 entries, 0 to 541908
Data columns (total 6 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   Description  540455 non-null  object 
 1   Quantity     541909 non-null  int64  
 2   InvoiceDate  541909 non-null  object 
 3   UnitPrice    541909 non-null  float64
 4   CustomerID   406829 non-null  float64
 5   Country      541909 non-null  object 
dtypes: float64(2), int64(1), object(3)
memory usage: 24.8+ MB


(None,
                            Description  Quantity     InvoiceDate  UnitPrice  \
 0   WHITE HANGING HEART T-LIGHT HOLDER         6  12/1/2010 8:26       2.55   
 1                  WHITE METAL LANTERN         6  12/1/2010 8:26       3.39   
 2       CREAM CUPID HEARTS COAT HANGER         8  12/1/2010 8:26       2.75   
 3  KNITTED UNION FLAG HOT WATER BOTTLE         6  12/1/2010 8:26       3.39   
 4       RED WOOLLY HOTTIE WHITE HEART.         6  12/1/2010 8:26       3.39   
 
    CustomerID         Country  
 0     17850.0  United Kingdom  
 1     17850.0  United Kingdom  
 2     17850.0  United Kingdom  
 3     17850.0  United Kingdom  
 4     17850.0  United Kingdom  )

# Анализ

## Подготовка к анализу

Датасет содержит транзакции с тремя столбцами:  
1. Member_number (номер покупателя)
2. Date (дата транзакции)
3. itemDescription (наименование товара)
  
Для выполнения анализа ассоциативных правил, нам нужно преобразовать эти данные в формат, который покажет, какие товары были куплены вместе в каждой транзакции

## Группировка данных

Преобразуем данные так, чтобы каждая строка представляла одну транзакцию, а столбцы показывали наличие или отсутствие товара (бинарное представление:   1 — куплен, 0 — не куплен).

In [7]:
data

,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,WHITE HANGING HEART T-LIGHT HOLDER,6,12/1/2010 8:26,2.55,17850.0,United Kingdom
1,WHITE METAL LANTERN,6,12/1/2010 8:26,3.39,17850.0,United Kingdom
2,CREAM CUPID HEARTS COAT HANGER,8,12/1/2010 8:26,2.75,17850.0,United Kingdom
3,KNITTED UNION FLAG HOT WATER BOTTLE,6,12/1/2010 8:26,3.39,17850.0,United Kingdom
4,RED WOOLLY HOTTIE WHITE HEART.,6,12/1/2010 8:26,3.39,17850.0,United Kingdom
...,...,...,...,...,...,...
541904,PACK OF 20 SPACEBOY NAPKINS,12,12/9/2011 12:50,0.85,12680.0,France
541905,CHILDREN'S APRON DOLLY GIRL,6,12/9/2011 12:50,2.10,12680.0,France
541906,CHILDRENS CUTLERY DOLLY GIRL,4,12/9/2011 12:50,4.15,12680.0,France
541907,CHILDRENS CUTLERY CIRCUS PARADE,4,12/9/2011 12:50,4.15,12680.0,France


In [8]:
data = data[data['Quantity'] > 0]
data

,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,WHITE HANGING HEART T-LIGHT HOLDER,6,12/1/2010 8:26,2.55,17850.0,United Kingdom
1,WHITE METAL LANTERN,6,12/1/2010 8:26,3.39,17850.0,United Kingdom
2,CREAM CUPID HEARTS COAT HANGER,8,12/1/2010 8:26,2.75,17850.0,United Kingdom
3,KNITTED UNION FLAG HOT WATER BOTTLE,6,12/1/2010 8:26,3.39,17850.0,United Kingdom
4,RED WOOLLY HOTTIE WHITE HEART.,6,12/1/2010 8:26,3.39,17850.0,United Kingdom
...,...,...,...,...,...,...
541904,PACK OF 20 SPACEBOY NAPKINS,12,12/9/2011 12:50,0.85,12680.0,France
541905,CHILDREN'S APRON DOLLY GIRL,6,12/9/2011 12:50,2.10,12680.0,France
541906,CHILDRENS CUTLERY DOLLY GIRL,4,12/9/2011 12:50,4.15,12680.0,France
541907,CHILDRENS CUTLERY CIRCUS PARADE,4,12/9/2011 12:50,4.15,12680.0,France


In [9]:
cleaned_data = data.drop(columns=['Quantity', 'UnitPrice', 'Country'])
cleaned_data['InvoiceDate'] = pd.to_datetime(cleaned_data['InvoiceDate'])
cleaned_data

,Description,InvoiceDate,CustomerID
0,WHITE HANGING HEART T-LIGHT HOLDER,2010-12-01 08:26:00,17850.0
1,WHITE METAL LANTERN,2010-12-01 08:26:00,17850.0
2,CREAM CUPID HEARTS COAT HANGER,2010-12-01 08:26:00,17850.0
3,KNITTED UNION FLAG HOT WATER BOTTLE,2010-12-01 08:26:00,17850.0
4,RED WOOLLY HOTTIE WHITE HEART.,2010-12-01 08:26:00,17850.0
...,...,...,...
541904,PACK OF 20 SPACEBOY NAPKINS,2011-12-09 12:50:00,12680.0
541905,CHILDREN'S APRON DOLLY GIRL,2011-12-09 12:50:00,12680.0
541906,CHILDRENS CUTLERY DOLLY GIRL,2011-12-09 12:50:00,12680.0
541907,CHILDRENS CUTLERY CIRCUS PARADE,2011-12-09 12:50:00,12680.0


In [10]:
basket = data.groupby(['CustomerID', 'InvoiceDate'])['Description'].apply(lambda x: ', '.join(x)).reset_index()

basket

,CustomerID,InvoiceDate,Description
0,12346.0,1/18/2011 10:01,MEDIUM CERAMIC TOP STORAGE JAR
1,12347.0,1/26/2011 14:30,"PINK NEW BAROQUECANDLESTICK CANDLE, BLUE NEW B..."
2,12347.0,10/31/2011 12:25,"MINI LIGHTS WOODLAND MUSHROOMS, PINK GOOSE FEA..."
3,12347.0,12/7/2010 14:57,"BLACK CANDELABRA T-LIGHT HOLDER, AIRLINE BAG V..."
4,12347.0,12/7/2011 15:52,"CLASSIC CHROME BICYCLE BELL , BICYCLE PUNCTURE..."
...,...,...,...
18439,18283.0,7/14/2011 13:20,GIN AND TONIC MUG
18440,18283.0,9/5/2011 12:35,"RECYCLING BAG RETROSPOT , 60 TEATIME FAIRY CAK..."
18441,18287.0,10/12/2011 10:23,"HAND WARMER OWL DESIGN, SET OF 3 WOODEN SLEIGH..."
18442,18287.0,10/28/2011 9:29,"SET OF 3 WOODEN SLEIGH DECORATIONS, PAINTED ME..."


In [11]:
basket_items = basket['Description'].str.get_dummies(sep=', ')
basket_items = basket_items.astype(bool)

basket_items

,4 PURPLE FLOCK DINNER CANDLES,50'S CHRISTMAS GIFT BAG LARGE,DOLLY GIRL BEAKER,I LOVE LONDON MINI BACKPACK,I LOVE LONDON MINI RUCKSACK,NINE DRAWER OFFICE TIDY,OVAL WALL MIRROR DIAMANTE,RED SPOT GIFT BAG LARGE,SET 2 TEA TOWELS I LOVE LONDON,SPACEBOY BABY GIFT SET,...,ZINC STAR T-LIGHT HOLDER,ZINC SWEETHEART SOAP DISH,ZINC SWEETHEART WIRE LETTER RACK,ZINC T-LIGHT HOLDER STAR LARGE,ZINC T-LIGHT HOLDER STARS LARGE,ZINC T-LIGHT HOLDER STARS SMALL,ZINC TOP 2 DOOR WOODEN SHELF,ZINC WILLIE WINKIE CANDLE STICK,ZINC WIRE KITCHEN ORGANISER,ZINC WIRE SWEETHEART LETTER TRAY
0,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
1,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
2,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
3,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
4,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18439,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
18440,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
18441,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
18442,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


In [54]:
item_count = basket_items.sum(axis=1)
item_count.max()

543

## Применение Apriori

Алгоритм Apriori основан на принципе "априорности", который утверждает, что если набор элементов (например, товары) является частым, то все его подмножества также будут частыми. Это позволяет исключить ненужные кандидаты на ранних этапах обработки данных.  

**Этапы составления правил:**  
1. Генерация частых наборов  
Алгоритм начинается с поиска одноэлементных частых наборов. Затем, на каждом следующем шаге, он генерирует кандидаты для наборов большего размера, используя частые наборы предыдущего уровня  
2. Подсчет поддержки  
Для каждого кандидата подсчитывается его поддержка, и только те, которые соответствуют заданному порогу (min_support), остаются для следующего шага.  
3. Генерация правил  
После того как все частые наборы были найдены, алгоритм генерирует ассоциативные правила, используя заданный порог достоверности (confidence).  

In [14]:
from mlxtend.frequent_patterns import apriori, association_rules

frequent_itemsets = apriori(basket_items, min_support=0.01, use_colnames=True, low_memory=True)


frequent_itemsets

,support,itemsets
0,0.010410,( SET 2 TEA TOWELS I LOVE LONDON )
1,0.013121,(10 COLOUR SPACEBOY PEN)
2,0.010139,(12 MESSAGE CARDS WITH ENVELOPES)
3,0.014964,(12 PENCIL SMALL TUBE WOODLAND)
4,0.016591,(12 PENCILS SMALL TUBE RED RETROSPOT)
...,...,...
979,0.014368,"(REGENCY CAKESTAND 3 TIER, ROSES REGENCY TEACU..."
980,0.010085,"(POPPY'S PLAYHOUSE KITCHEN, POPPY'S PLAYHOUSE ..."
981,0.010030,"(REGENCY TEA PLATE ROSES , REGENCY TEA PLATE G..."
982,0.012958,"(PINK REGENCY TEACUP AND SAUCER, ROSES REGENCY..."


In [15]:
rules = association_rules(frequent_itemsets, metric="confidence", min_threshold=0.01)

rules

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,zhangs_metric
0,(PACK OF 72 RETROSPOT CAKE CASES),(60 CAKE CASES DOLLY GIRL DESIGN),0.055791,0.019193,0.010247,0.183673,9.569699,0.009176,1.201488,0.948416
1,(60 CAKE CASES DOLLY GIRL DESIGN),(PACK OF 72 RETROSPOT CAKE CASES),0.019193,0.055791,0.010247,0.533898,9.569699,0.009176,2.025759,0.913027
2,(SET OF 20 VINTAGE CHRISTMAS NAPKINS),(60 CAKE CASES VINTAGE CHRISTMAS),0.026458,0.025157,0.010085,0.381148,15.150615,0.009419,1.575243,0.959380
3,(60 CAKE CASES VINTAGE CHRISTMAS),(SET OF 20 VINTAGE CHRISTMAS NAPKINS),0.025157,0.026458,0.010085,0.400862,15.150615,0.009419,1.624904,0.958099
4,(72 SWEETHEART FAIRY CAKE CASES),(60 TEATIME FAIRY CAKE CASES),0.027163,0.035621,0.011982,0.441118,12.383525,0.011015,1.725549,0.944915
...,...,...,...,...,...,...,...,...,...,...
949,"(LUNCH BAG BLACK SKULL., LUNCH BAG RED RETROS...","(LUNCH BAG PINK POLKADOT, LUNCH BAG CARS BLUE)",0.027977,0.023151,0.010356,0.370155,15.988617,0.009708,1.550935,0.964437
950,(LUNCH BAG PINK POLKADOT),"(LUNCH BAG CARS BLUE, LUNCH BAG BLACK SKULL.,...",0.050477,0.014097,0.010356,0.205156,14.553433,0.009644,1.240373,0.980795
951,(LUNCH BAG CARS BLUE),"(LUNCH BAG PINK POLKADOT, LUNCH BAG BLACK SKU...",0.052375,0.016591,0.010356,0.197723,11.917631,0.009487,1.225772,0.966723
952,(LUNCH BAG BLACK SKULL.),"(LUNCH BAG PINK POLKADOT, LUNCH BAG CARS BLUE,...",0.057038,0.015018,0.010356,0.181559,12.089072,0.009499,1.203485,0.972765


### Тривиальные правила

Тривиальное правило — это правило ассоциативного анализа, которое не несет значимой информации или не открывает новых закономерностей. Обычно такие правила встречаются часто, но они очевидны и не дают полезного для бизнеса или исследований инсайта.  

**Признаки тривиальных правил:**  
1. Низкий лифт (Lift):  
Если значение Lift близко к 1, это говорит о том, что товары покупаются вместе примерно так же часто, как если бы они были независимы друг от друга. Это делает правило малополезным, так как не указывает на сильную взаимосвязь.  
Lift < 1 может указывать на то, что товары даже покупаются реже вместе, чем случайно.  
2. Высокая частота, но очевидная связь:  
Например, если правило выводит: «Если покупают хлеб, то покупают и воду», но эти товары и так всегда покупаются вместе по умолчанию, то такое правило будет считаться тривиальным, потому что это легко предсказать.  
  
Правило: "Если покупают молоко, то покупают воду".  
Confidence: 80% (покупатели часто покупают эти два товара вместе).  
Lift: 1.05 (они покупаются вместе чуть чаще, чем можно было бы ожидать случайно).  

In [16]:
rules[ np.abs(rules['lift'] - 1) < 0.32 ]

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,zhangs_metric
534,(REGENCY CAKESTAND 3 TIER),(WHITE HANGING HEART T-LIGHT HOLDER),0.092388,0.106864,0.01285,0.139085,1.301509,0.002977,1.037426,0.255243
535,(WHITE HANGING HEART T-LIGHT HOLDER),(REGENCY CAKESTAND 3 TIER),0.106864,0.092388,0.01285,0.120244,1.301509,0.002977,1.031663,0.259380


Все правила показывают *lift чуть выше 1*, что говорит о том, что товары *покупаются вместе чуть чаще, чем ожидалось случайно*.  
Confidence для данных правил составляет 13.9% и 12%, что означает, что вероятность покупки "последующего товара" при покупке "предшествующего товара" относительно низкая, но все же выше случайного уровня.

**Например:**  
(REGENCY CAKESTAND 3 TIER) → (WHITE HANGING HEART T-LIGHT HOLDER) и (WHITE HANGING HEART T-LIGHT HOLDER) → (REGENCY CAKESTAND 3 TIER):

Оба правила имеют lift 1.30. Это указывает на слабую, но все же существующую взаимосвязь между этими двумя товарами.
Низкий leverage (0.002977) говорит о том, что данное правило имеет минимальное влияние на выбор товаров.  
  
Leverage (или рычаг) в контексте ассоциативного анализа — это метрика, которая измеряет, насколько часто два товара покупаются вместе, по сравнению с тем, как часто они бы покупались независимо друг от друга. Это дает представление о том, насколько сильна ассоциация между предшествующим и последующим товарами.  
  
$
Leverage(A \rightarrow B) = Support(A \bigcap B) - (Support(A) \times Support(B))
$  

### Переберем разные уровни поддержки и достоверности

In [17]:
basket_items.shape

(18444, 3904)

In [18]:
basket_items_slim = basket_items[:9000]

basket_items_slim

,4 PURPLE FLOCK DINNER CANDLES,50'S CHRISTMAS GIFT BAG LARGE,DOLLY GIRL BEAKER,I LOVE LONDON MINI BACKPACK,I LOVE LONDON MINI RUCKSACK,NINE DRAWER OFFICE TIDY,OVAL WALL MIRROR DIAMANTE,RED SPOT GIFT BAG LARGE,SET 2 TEA TOWELS I LOVE LONDON,SPACEBOY BABY GIFT SET,...,ZINC STAR T-LIGHT HOLDER,ZINC SWEETHEART SOAP DISH,ZINC SWEETHEART WIRE LETTER RACK,ZINC T-LIGHT HOLDER STAR LARGE,ZINC T-LIGHT HOLDER STARS LARGE,ZINC T-LIGHT HOLDER STARS SMALL,ZINC TOP 2 DOOR WOODEN SHELF,ZINC WILLIE WINKIE CANDLE STICK,ZINC WIRE KITCHEN ORGANISER,ZINC WIRE SWEETHEART LETTER TRAY
0,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
1,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
2,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
3,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
4,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8995,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
8996,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,True,False,False,False,False
8997,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
8998,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


#### Фиксиурем поддержку и меняем достоверность

In [35]:
support_values = [0.01]
confidence_values = [0.5, 0.7, 0.9]
results = {}

for support in support_values:
    frequent_itemsets = apriori(basket_items_slim, min_support=support, use_colnames=True, low_memory=True)
    
    rules_for_support = {}
    for confidence in confidence_values:
        rules = association_rules(frequent_itemsets, metric="confidence", min_threshold=confidence)
        
        rules_for_support[confidence] = rules
    
    results[support] = rules_for_support

for support in results:
    for confidence in results[support]:
        print(f"Support: {support}, Confidence: {confidence}")
        print(f"Count of lift > 25: {(results[support][confidence]['lift'] > 25).shape[0]}")
        print(f"Mean of lift: {round(np.mean(results[support][confidence]['lift']), 3)}")
        print(f"Std of lift: {round(np.std(results[support][confidence]['lift']), 3)}")
        print(f'Size = {results[support][confidence].shape}')
        print('-' * 50)


Support: 0.01, Confidence: 0.5
Count of lift > 25: 276
Mean of lift: 21.286
Std of lift: 14.031
Size = (276, 10)
--------------------------------------------------
Support: 0.01, Confidence: 0.7
Count of lift > 25: 66
Mean of lift: 31.465
Std of lift: 15.998
Size = (66, 10)
--------------------------------------------------
Support: 0.01, Confidence: 0.9
Count of lift > 25: 6
Mean of lift: 46.13
Std of lift: 10.993
Size = (6, 10)
--------------------------------------------------


#### Фиксиурем достоверность и меняем поддержку 

In [36]:
support_values = [0.0085, 0.009, 0.01, 0.1]
confidence_values = [0.5]
results = {}

for support in support_values:
    frequent_itemsets = apriori(basket_items_slim, min_support=support, use_colnames=True, low_memory=True)
    
    rules_for_support = {}
    for confidence in confidence_values:
        rules = association_rules(frequent_itemsets, metric="confidence", min_threshold=confidence)
        
        rules_for_support[confidence] = rules
    
    results[support] = rules_for_support

for support in results:
    for confidence in results[support]:
        print(f"Support: {support}, Confidence: {confidence}")
        print(f"Count of lift > 25: {(results[support][confidence]['lift'] > 25).shape[0]}")
        print(f"Mean of lift: {round(np.mean(results[support][confidence]['lift']), 3)}")
        print(f"Std of lift: {round(np.std(results[support][confidence]['lift']), 3)}")
        print(f'Size = {results[support][confidence].shape}')
        print('-' * 50)


Support: 0.0085, Confidence: 0.5
Count of lift > 25: 449
Mean of lift: 21.709
Std of lift: 14.738
Size = (449, 10)
--------------------------------------------------
Support: 0.009, Confidence: 0.5
Count of lift > 25: 382
Mean of lift: 21.476
Std of lift: 14.098
Size = (382, 10)
--------------------------------------------------
Support: 0.01, Confidence: 0.5
Count of lift > 25: 276
Mean of lift: 21.286
Std of lift: 14.031
Size = (276, 10)
--------------------------------------------------
Support: 0.1, Confidence: 0.5
Count of lift > 25: 0
Mean of lift: nan
Std of lift: nan
Size = (0, 10)
--------------------------------------------------


#### Влияние параметров

* **Зависимость между поддержкой и количеством правил**  
Увеличение значений поддержки приводит к уменьшению количества извлеченных правил

* **Качество правил и лифт**  
Правила с высоким лифтом (более 25) свидетельствуют о значительных ассоциациях между товарами. Высокие значения лифта при низкой поддержке показывают, что ассоциации могут быть полезными  

* **Компромисс между количеством и качеством**  
Существует компромисс между количеством и качеством ассоциативных правил. Более строгие параметры (высокая поддержка и высокая достоверность) могут улучшить качество правил (повышая лифт), но при этом сокращают количество извлеченных правил.

## FPG

FPG устраняет необходимость многократного прохода по данным, используя структуру дерева (FP-tree) для хранения частых элементов и их связей. Вместо генерации кандидатов, FPG фокусируется на сокращении пространства поиска, сохраняя только те элементы, которые поддерживают заданный порог  

**Этапы составления правил:**
1. Создание FP-tree  
Алгоритм сначала строит FP-дерево, которое содержит все частые элементы и их частоту, это происходит за один проход по данным  
2. Разделение дерева  
Дерево разбивается на условные поддеревья для каждого элемента, что более эффективно  
3. Извлечение частых наборов  
Затем алгоритм извлекает частые наборы из FP-дерева, используя условные поддеревья  

In [40]:
from mlxtend.frequent_patterns import fpgrowth

frequent_itemsets = fpgrowth(basket_items, min_support=0.01, use_colnames=True)

frequent_itemsets


,support,itemsets
0,0.010573,(MEDIUM CERAMIC TOP STORAGE JAR)
1,0.092388,(REGENCY CAKESTAND 3 TIER)
2,0.047549,(ALARM CLOCK BAKELIKE RED )
3,0.042778,(ALARM CLOCK BAKELIKE GREEN)
4,0.035621,(60 TEATIME FAIRY CAKE CASES)
...,...,...
979,0.011332,"(HAND WARMER UNION JACK, HAND WARMER OWL DESIGN)"
980,0.010301,"(HAND WARMER UNION JACK, HAND WARMER SCOTTY DO..."
981,0.012687,"(KEY FOB , BACK DOOR )"
982,0.015235,"(LARGE WHITE HEART OF WICKER, SMALL WHITE HEAR..."


In [41]:
from mlxtend.frequent_patterns import association_rules

rules = association_rules(frequent_itemsets, metric="confidence", min_threshold=0.1)

rules


,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,zhangs_metric
0,(REGENCY CAKESTAND 3 TIER),(WHITE HANGING HEART T-LIGHT HOLDER),0.092388,0.106864,0.012850,0.139085,1.301509,0.002977,1.037426,0.255243
1,(WHITE HANGING HEART T-LIGHT HOLDER),(REGENCY CAKESTAND 3 TIER),0.106864,0.092388,0.012850,0.120244,1.301509,0.002977,1.031663,0.259380
2,(ALARM CLOCK BAKELIKE RED ),(ALARM CLOCK BAKELIKE GREEN),0.047549,0.042778,0.028790,0.605473,14.153799,0.026756,2.426253,0.975744
3,(ALARM CLOCK BAKELIKE GREEN),(ALARM CLOCK BAKELIKE RED ),0.042778,0.047549,0.028790,0.673004,14.153799,0.026756,2.912727,0.970880
4,(PACK OF 60 PINK PAISLEY CAKE CASES),(60 TEATIME FAIRY CAKE CASES),0.036272,0.035621,0.015181,0.418535,11.749561,0.013889,1.658533,0.949324
...,...,...,...,...,...,...,...,...,...,...
945,(BACK DOOR ),(KEY FOB ),0.012687,0.020332,0.012687,1.000000,49.184000,0.012429,inf,0.992257
946,(LARGE WHITE HEART OF WICKER),(SMALL WHITE HEART OF WICKER),0.027705,0.031555,0.015235,0.549902,17.426796,0.014361,2.151632,0.969477
947,(SMALL WHITE HEART OF WICKER),(LARGE WHITE HEART OF WICKER),0.031555,0.027705,0.015235,0.482818,17.426796,0.014361,1.879985,0.973331
948,(HEART OF WICKER LARGE),(LARGE WHITE HEART OF WICKER),0.043429,0.027705,0.010410,0.239700,8.651729,0.009207,1.278831,0.924569


In [51]:
rules[rules['lift'] < 1.4]

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,zhangs_metric
0,(REGENCY CAKESTAND 3 TIER),(WHITE HANGING HEART T-LIGHT HOLDER),0.092388,0.106864,0.01285,0.139085,1.301509,0.002977,1.037426,0.255243
1,(WHITE HANGING HEART T-LIGHT HOLDER),(REGENCY CAKESTAND 3 TIER),0.106864,0.092388,0.01285,0.120244,1.301509,0.002977,1.031663,0.259380


Основное различие между алгоритмами Apriori и FPG (Frequent Pattern Growth) заключается в том, как они обрабатывают данные и хранят результаты.

# Выводы

1. Метод ассоциативных правил позволяет выявлять взаимосвязи между элементами в больших наборах данных

2. В ходе экспериментов с изменением параметров (минимальная поддержка и достоверность) было замечено, что увеличение этих значений приводит к уменьшению общего количества извлеченных правил. Это подтверждает, что более строгие параметры отфильтровывают менее значимые ассоциации

3. Оценка значений метрик достоверности и лифта показала, что правила с высоким значением лифта (> 1) указывают на значительное усиление вероятности покупки одного товара при покупке другого

4. Сравнение алгоритмов Apriori и FPG продемонстрировало, что FPG более эффективен для больших наборов данных благодаря своей структуре FP-дерева, что позволяет избежать генерации большого количества кандидатов и снизить количество проходов по данным
